In [12]:
# Import libraries
import os
os.chdir(r"C:\\Users\\Dell #050\\Documents\\MABe")
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter
import pandas as pd
from tqdm import tqdm

In [9]:
# Read annotation files
annotation_files = glob(os.path.join("datasets", "train_annotation","**", "*.parquet"))
print(f"Number of annotation files: {len(annotation_files)}")

Number of annotation files: 847


In [10]:
# Read a sample annotation file
sample_data = pd.read_parquet(annotation_files[0])
sample_data.head()

,agent_id,target_id,action,start_frame,stop_frame
0,1,3,chase,2,54
1,1,3,chase,128,234
2,3,2,avoid,324,342
3,3,1,avoid,324,342
4,1,2,chase,942,1052


In [13]:
# Count frequency of each action
def process_file(file_path):
    """Reads one file and returns a Counter of action frequencies."""
    data = pd.read_parquet(file_path)
    counter = Counter()
    for row in data.itertuples():
        counter[row.action] += (row.stop_frame - row.start_frame) + 1
    return counter

# Initialize global counter
action_freq_map = Counter()

# Use multithreading to process multiple files in parallel
with ThreadPoolExecutor(max_workers=8) as executor:  # adjust max_workers as needed
    futures = {executor.submit(process_file, f): f for f in annotation_files}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing annotation files"):
        action_freq_map.update(future.result())

# Convert back to normal dict if desired
action_freq_map = dict(action_freq_map)
action_freq_map

Processing annotation files: 100%|██████████| 847/847 [00:00<00:00, 69326.66it/s]


{'rear': 235297,
 'approach': 89238,
 'avoid': 26204,
 'attack': 526332,
 'chase': 27530,
 'submit': 8564,
 'chaseattack': 5537,
 'sniff': 2188427,
 'mount': 287678,
 'intromit': 348655,
 'sniffgenital': 711793,
 'sniffbody': 106792,
 'reciprocalsniff': 42174,
 'dominancemount': 17966,
 'attemptmount': 6161,
 'escape': 92375,
 'defend': 91391,
 'sniffface': 74338,
 'shepherd': 29652,
 'selfgroom': 83117,
 'dig': 79897,
 'allogroom': 7625,
 'dominancegroom': 4777,
 'follow': 40377,
 'climb': 61344,
 'run': 1808,
 'rest': 87806,
 'huddle': 24447,
 'genitalgroom': 6320,
 'exploreobject': 3783,
 'flinch': 1861,
 'biteobject': 2359,
 'tussle': 5178,
 'ejaculate': 1614,
 'freeze': 31765,
 'dominance': 38483,
 'disengage': 12300}

Let's create a map where action will be stored such a way that each action will have a list of dicts, key will be file name or path and the value will be a list of (start frame, stop frame) where the action is found.

The frequency map should also include 'no action'.

The dataset split should be done such a way that it will traverse all the files. Number of frames for each action should be predefined. Now, a segment should be fetched randomly to create the dataset.
- Segment size should be decided from the average action duration distribution.
- Some rebalnce method is needed to handle overlapping actions which can make the sampled data improper.
- For each video file action map should be prepared and stored in json for reference.
- For each action a map should be prepared for non-overlapping action segment which can be used for oversampling.